# k-Nearest Neighbors

***

## Índice

1. [Importando bibliotecas](#importando-bibliotecas)
2. [Treinamento do modelo](#treinamento-do-modelo)
3. [Impacto do tamanho da vizinhança](#impacto-do-tamanho-da-vizinhança)
4. [Personalizando a medida de distância](#personalizando-a-medida-de-distância)

## Importando bibliotecas

In [6]:
# Bibliotecas de manipualção e visualização de dados
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

# Classes do modelo de aprendizado
from sklearn.neighbors import KNeighborsClassifier

# Funções de avaliação dos modelos
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Análise do conjunto de dados

In [ ]:
#carregando o csv
dataset = pd.read_csv()

In [33]:
# carregando o dataset
dataset = load_iris(as_frame=True) 

print("Quantas classes existem nesse dataset?\n%d" %(len(dataset["target"].unique())))
print("\nQuantas instâncias existem no dataset?\n%d" %(len(dataset["data"])))
print("\nQuantas features existem no dataset?\n%d" %(len(dataset[])))
# print("\nQue features são essas?\n%s" %  list((X.columns)))
# print("\nQual o numero de instâncias por classe?")
# frequency = np.unique(y.values, return_counts=True)[1]
# for c in range(3):
#     print("Classe %d: %d" %(c, frequency[c]))



Quantas classes existem nesse dataset?
3

Quantas instâncias existem no dataset?
150


## Treinamento do modelo

In [ ]:
# carregando o dataset
X, y = load_wine(return_X_y=True, as_frame=True)

# vamos escolher apenas classes do dataset
class_a = 0
class_b = 1
class_0_instances = (y == class_a)
class_1_instances = (y == class_b)

filtered_y = y[class_0_instances | class_1_instances]
filtered_X = X[class_0_instances | class_1_instances]

# cores e simbolos para as classses
colors = {0: "steelblue", 1: "darkorange", 2: "mediumseagreen"}
markers = {0: "s", 1: "^", 2:"o"}

# vamos observar as duas features
feature_0 = "alcohol"
feature_1 = "color_intensity"


In [ ]:
# dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(filtered_X[[feature_0, feature_1]], filtered_y, test_size=0.3, random_state=199)

# vamos criar um classificador kNN com k=11
model = KNeighborsClassifier(n_neighbors=11)
model.fit(X_train, y_train)

# e ver a sua performance no dataset de teste
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
# vamos fazer uma pequena modificação na nossa função de gerar a região de decisão, agora vamos incluir o tamanho da vizinhança para o kNN
def show_decision_region(x, y, clf, f0, f1):
    plot_decision_regions(x, y, clf=clf)
    plt.xlabel(f0)
    plt.ylabel(f1)
    if clf.__class__.__name__ == "KNeighborsClassifier":
        plt.title(clf.__class__.__name__ + " k = " + str(clf.n_neighbors))
    else:
        plt.title(clf.__class__.__name__)
    plt.show()

show_decision_region(
    np.array(
        [
            X_test[feature_0].values, 
            X_test[feature_1].values,
        ]
    ).T, 
    y_test.values, 
    model, 
    feature_0, 
    feature_1
)

## Impacto do tamanho da vizinhança

Vamos voltar um pouco e rever como o dataset de treino se comporta.

In [ ]:
plt.scatter(
    X_train[feature_0][class_0_instances],
    X_train[feature_1][class_0_instances], 
    c=colors[class_a], 
    marker=markers[class_a]
)
plt.scatter(
    X_train[feature_0][class_1_instances], 
    X_train[feature_1][class_1_instances], 
    c=colors[class_b], 
    marker=markers[class_b]
)

Vemos que a classe triangulo laranja tem alguns exemplares bem próximos dos quadrados azuis. O que acontece se mudarmos o tamanho da vizinhança no nosso classificador?

Agora vamos observar a região de decisão, reduzindo o tamanho da vizinhança.

A medida em que a quantidade de vizinhos reduz, a região de decisão fica menos suave. Isso porque quanto menos vizinhos para decidir se uma instancia é de uma classe ou não, maior a sensibilidade do algoritmo.

Decidir o valor ideal para esse hiperparâmetro é depende de problema para problema. Para alguns datasets, um valor menor de vizinhos pode ser beneficial para diferenciar padrões com baixa ocorrência ou que estão próximos de outras classe com maior frequência. Entretanto, quanto menor a quantidade de vizinhos, maior é a chance do algoritmo ser afetado por ruído.

In [ ]:
for k in [11, 9, 7, 5, 3, 1]:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)

    # e ver a sua performance no dataset de teste
    show_decision_region(
        np.stack(
            [
                X_test[feature_0].values, 
                X_test[feature_1].values,
            ],
            axis=1
        ), 
        y_test.values, 
        model, 
        feature_0, 
        feature_1
    )

## Personalizando a medida de distância

Também podemos mudar a métrica utilizada para calcular a distância entre as amostras.

O hiperparâmetro _metric_ pode assumir dois tipos diferentes, uma _string_ ou uma função. Caso o valor seja uma _string_, as possíveis funções de distâncias estão presentes [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric). Entretanto, você tamém pode criar uma função que calcula a distância entre métricas.

Vamos definir duas distâncias diferentes, a distância Euclidiana e a distância Manhattan:

- Manhattan: $D_M(x, y) = |x_1-x_2| + |y_1-y_2|$
- Euclidiana: $D(x, y) = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}$

In [ ]:
# vamos implementar a distância manhattan, com pesos diferentes para cada uma das features
def knn_custom_distance(x, y, weights=np.array([2, 1])):
    return (abs(x - y)*weights).sum()

model = KNeighborsClassifier(n_neighbors=11, metric=knn_custom_distance)
model.fit(X_train, y_train)

# e ver a sua performance no dataset de test
print(classification_report(y_test, model.predict(X_test)))

show_decision_region(
    np.array(
        [
            X_test[feature_0].values, 
            X_test[feature_1].values,
        ]
    ).T, 
    y_test.values, 
    model, 
    feature_0, 
    feature_1
)

O comportamento padrão do algoritmo atribui pesos iguais para as distâncias entre a instância de teste e as de treino. Entretanto, podemos atribuir pesos diferentes em função das distâncias. Isso pode ser feito mudando o hiperparâmetro _weights_ de três formas diferentes: 

- _"uniform"_ atribui o rótulo da classe majoritária dos k vizinhos da instancia de teste.
- _"distance"_ utiliza como peso o inverso da distância ($1/d$), dando peso menor instâncias mais longe da instancia de teste.
- E uma _função customizada_, onde você pode definir a função para atribuir pesos diferentes para um vetor de distâncias.

In [ ]:
# vamos criar um classificador kNN com k=11
model = KNeighborsClassifier(n_neighbors=11, weights="distance")
model.fit(X_train, y_train)

# e ver a sua performance no dataset de teste
print(classification_report(y_test, model.predict(X_test)))

show_decision_region(
    np.array(
        [
            X_test[feature_0].values, 
            X_test[feature_1].values,
        ]
    ).T, 
    y_test.values, 
    model, 
    feature_0, 
    feature_1
)